Ejercicio práctico de detección de personas usando barbijo o no.

Idea de https://www.youtube.com/watch?v=IOI0o3Cxv9Q

Lo adapte usando Colab.

Otros links utulies:
* https://medium.com/swlh/tensorflow-2-object-detection-api-with-google-colab-b2af171e81cc
* https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/tensorflow-1.14/training.html (pero no es en colab)
* https://www.geeksforgeeks.org/python-opencv-cv2-rotate-method/

# Conectar al Drive

In [ ]:
#Para conectar con el Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Para ubicarme en la carpeta de trabajo donde voy a trabajar
%cd /content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla

Las carpetas que hay son:
* **script:** contiene generate_tfrecord, que es del modelo oficial de tensorflow.
* **workspace** contiene:
  - **annotations:** van las etiquetas del modelo
  - **images:** van las imagenes de entrenamiento y test (usadas como validación)
  - **models:** donde van archivos relacionados al modelo.
  - **pre-trained-models:** contiene el modelo preentrenado. En este caso en un detector simple que usa Mobilnet v2, 320x320 pixels.

# Instalaciones

In [ ]:
#Para ver si hay GPU
import tensorflow as tf
print(tf.__version__)

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
# Librerías a usar
import cv2 #Import opencv
import os
import time

### Instalar paquetes de Object Detection de Tensorflow

In [ ]:
# Usar esta línea si ya se ejecutó previamente la Colab
!rm -rf ./models/

# Para clonar Tensorflow Model Garden
!git clone --depth 1 https://github.com/tensorflow/models/

In [ ]:
# Compile the Object Detection API protocol buffers
# %%bash
# cd models/research/
# protoc object_detection/protos/*.proto --python_out=.
# cp object_detection/packages/tf2/setup.py -
# python -m pip install .
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .


# Tomar las fotos para el data Set

Colab aparte

# Preparación de los data para el entrenamiento

Lo hice con labelimg y guarde los .xml en drive.

# Seteo de Paths


In [ ]:
WORKSPACE_PATH = './Tensorflow/workspace'
SCRIPTS_PATH = './Tensorflow/scripts'
APIMODEL_PATH = './Tensorflow/models'
ANNOTATION_PATH = './Tensorflow/workspace/annotations'
IMAGE_PATH = './Tensorflow/workspace/images'
MODEL_PATH = './Tensorflow/workspace/models'
PRETRAINED_MODEL_PATH = './Tensorflow/workspace/pre-trained-models'
CONFIG_PATH = './Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = './Tensorflow/workspace/models/my_ssd_mobnet/'

# Crear Label Map

In [ ]:
#Crea un .pbtxt con las etiquetas
labels = [{'name':'Mask', 'id':1}, {'name':'NoMask', 'id':2}]

with open(ANNOTATION_PATH + '/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# Crear TF records

In [ ]:
# crea los TF records para entrenamiento y testeo (validación)
# !python generate_tfrecord.py -x '[ruta_a_la_carpeta_de_entrenamiento]' -l '[ruta_a_la_carpeta_de_anotaciones]/label_map.pbtxt' -o '[ruta_a_la_carpeta_de_anotaciones]/entrenar.record'
# !python generate_tfrecord.py -x '[ruta_a_la_carpeta_de_prueba]' -l '[ruta_a_la_carpeta_de_anotaciones]/label_map.pbtxt' -o '[ruta_a_la_carpeta_de_anotaciones]/test.record'

!python '/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/scripts/generate_tfrecord.py' -x '/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/images/train' -l '/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/annotations/label_map.pbtxt' -o '/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/annotations/train.record'
!python '/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/scripts/generate_tfrecord.py' -x'/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/images/test' -l '/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/annotations/label_map.pbtxt' -o '/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/annotations/test.record'


# 4. Copiar Model Config a la Carpeta de entrenamiento

In [ ]:
!mkdir {'Tensorflow/workspace/models/my_ssd_mobnet'}
!cp {'./Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {'./Tensorflow/workspace/models/my_ssd_mobnet'}

# 5. Actualizar Config para Transfer Learning

In [ ]:
#import tensorflow as tf

from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
CONFIG_PATH = './Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config'

In [ ]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [ ]:
#Config original
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 1
pipeline_config.train_config.fine_tune_checkpoint = '/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= '/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/annotations/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = ['/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/annotations/train.record']
pipeline_config.eval_input_reader[0].label_map_path = '/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/annotations/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = ['/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/annotations/test.record']

In [ ]:
#config modificado
config

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Entrenamiento del modelo

In [ ]:
# Si aparece error: ImportError: cannot import name '_registerMatType' from 'cv2.cv2'
#desinstalar OpenCv
!pip uninstall opencv-python -y
#Instalar otra versión
!pip install opencv-python-headless==4.1.2.30 

In [ ]:
#si sale DNN library is not found en el entrenamiento
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
#Entrenamiento
%cd '/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla'
!python '/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/models/research/object_detection/model_main_tf2.py' \
--pipeline_config_path='/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config' \
--model_dir='/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/models/my_ssd_mobnet' \
--num_train_steps=5000

# 7. Guardar el modelo entrenado desde el Checkpoint

In [ ]:
#Si da error ImportError: cannot import name 'BaseImageAugmentationLayer' from 'keras.layers.preprocessing.image_preprocessing' 
!pip uninstall tensorflow -y
!pip install  tensorflow==2.8
print(tf.__version__)

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial() #Aca se pone el numero del último ckpt del entrenamiento
#/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/Tensorflow/workspace/models/my_ssd_mobnet/ckpt-6.index


@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detección en tiempo real

In [ ]:
import cv2 
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')
category_index

In [ ]:
#Dar acceso a la cámara. En este caso pasa un video.
cap = cv2.VideoCapture('/content/drive/MyDrive/05-DetecciónObjetosEjercicio/Mascarilla/VID_20220528_175356981.mp4')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_output = cv2.VideoWriter('output6.mp6',fourcc, 10.0, (width,height)) #Poner el nombre del video de salida



In [ ]:
print((width,height))

In [ ]:

while True: 

    ret, frame = cap.read() #Toma cada frame de cap
    image_np = np.array(frame) #Convierte en numpy array
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32) #Convierte en tensor
    detections = detect_fn(input_tensor) #Usa la función detect_fn para hacer la detección y le pasa el input antes definido
    
    #Preproceso
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    #Para visualizar el box con la información
    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=1, #Cantidad de box en el frame
                min_score_thresh=.5,
                agnostic_mode=False,
                line_thickness=8) #Grueso del box. Por defecto 4
    
    image_np_with_detections = cv2.resize(image_np_with_detections, (width,height))
    video_output.write(image_np_with_detections)
  
    
    cv2_imshow(image_np_with_detections)
   
    
    if cv2.waitKey(1) & 0xFF == ord('q'): #Para le proceso
        cap.release() #Remueve el input video de la memoria
        break